<a href="https://colab.research.google.com/github/FacuML/NLP/blob/main/002_Parte/011/003_Tokens_y_Embeddings_de_Tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

💡 **NOTA**: Necesitaremos usar una GPU para ejecutar los ejemplos en este cuaderno. En Google Colab, andá a
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [ ]:
# %%capture
!pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

# Descargando y Ejecutando un LLM

El primer paso es cargar nuestro modelo en la GPU para una inferencia más rápida. Tené en cuenta que cargamos el modelo y el tokenizador por separado para poder explorarlos individualmente.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargar modelo y tokenizador
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Ejemplo en español
prompt = "Escribí un email disculpándote con María por el incidente en el asado del domingo. Explicá cómo sucedió.<|assistant|>"

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generar el texto
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Imprimir la salida
print(tokenizer.decode(generation_output[0]))

ModuleNotFoundError: Could not import module 'AutoModelForCausalLM'. Are this object's requirements defined correctly?

# Comparando Tokenizadores de LLMs Entrenados

En esta sección, vamos a ver cómo diferentes modelos tokenizan el texto. Es interesante ver cómo cada modelo maneja el español, los emojis y diferentes tipos de texto.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [ ]:
# Texto de ejemplo adaptado al español y contexto argentino
text = """
Español y MAYÚSCULAS
🧉 ñ
show_tokens False None elif == >= else: dos tabs:"    " Tres tabs: "       "
12.0*50=600
El mate está muy caliente
"""

In [ ]:
# Probamos diferentes tokenizadores
show_tokens(text, "bert-base-uncased")

In [ ]:
show_tokens(text, "bert-base-cased")

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

In [ ]:
show_tokens(text, "gpt2")

# Embeddings de Palabras Contextualizados desde un Modelo de Lenguaje (Como BERT)

Los embeddings contextualizados nos permiten obtener representaciones numéricas de palabras que tienen en cuenta el contexto en el que aparecen.

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Cargar tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Cargar modelo de lenguaje
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenizar la oración
tokens = tokenizer('Hola mundo', return_tensors='pt')

# Procesar los tokens
output = model(**tokens)[0]

print("Forma del tensor de salida:")
print(output.shape)

print("\nTokens individuales:")
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

# Embeddings de Texto (Para Oraciones y Documentos Completos)

Los embeddings de texto nos permiten convertir oraciones completas en vectores numéricos, útiles para búsqueda semántica y análisis de similitud.

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar modelo
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convertir texto a embeddings
vector = model.encode("¡La mejor película que vi!")

print("Dimensiones del vector:", vector.shape)